In [7]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}
 
# from 2018 season
url = 'https://www.footballdb.com/statistics/nfl/player-stats/kicking/2018/regular-season?sort=kickpts&limit=100'
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')

print(soup)

In [17]:
# content
content = soup.find(id='leftcol')
# headers
headers = content.find_all('th')
# getting column names
cols = [head.text.strip() for head in headers if head.text.strip()]
print(cols)

['Player', 'Team', 'Gms', 'PAT', 'FG', '0-19', '20-29', '30-39', '40-49', '50+', 'Lg', 'Pts']


In [24]:
import pandas as pd
# stats
table = content.find_all('tr')
# converting to dataframe
res = []
for tr in table:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
# data
data = pd.DataFrame(res, columns=cols)
data.head()

,Player,Team,Gms,PAT,FG,0-19,20-29,30-39,40-49,50+,Lg,Pts
0,Ka'imi FairbairnK. Fairbairn,HOU,16,39/41,37/42,1/1,9/9,11/11,12/15,4/6,54,150
1,Justin TuckerJ. Tucker,BAL,16,36/37,35/39,0/0,12/12,9/9,9/11,5/7,56,141
2,Harrison ButkerH. Butker,KC,16,65/69,24/27,0/0,9/9,7/7,6/7,2/4,54,137
3,Wil LutzW. Lutz,NO,16,52/53,28/30,1/1,5/5,9/9,11/12,2/3,54,136
4,Stephen GostkowskiS. Gostkowski,NE,16,49/50,27/32,0/0,11/11,10/10,4/6,2/5,52,130


In [42]:
# checking element type
data["Gms"] = pd.to_numeric(data["Gms"])
data["Lg"] = pd.to_numeric(data["Lg"])
data["Pts"] = pd.to_numeric(data["Pts"])

data.head()

,Player,Team,Gms,PAT,FG,0-19,20-29,30-39,40-49,50+,Lg,Pts
0,Ka'imi FairbairnK. Fairbairn,HOU,16,39/41,37/42,1/1,9/9,11/11,12/15,4/6,54,150
1,Justin TuckerJ. Tucker,BAL,16,36/37,35/39,0/0,12/12,9/9,9/11,5/7,56,141
2,Harrison ButkerH. Butker,KC,16,65/69,24/27,0/0,9/9,7/7,6/7,2/4,54,137
3,Wil LutzW. Lutz,NO,16,52/53,28/30,1/1,5/5,9/9,11/12,2/3,54,136
4,Stephen GostkowskiS. Gostkowski,NE,16,49/50,27/32,0/0,11/11,10/10,4/6,2/5,52,130


In [48]:
def split(csv,col):
  # splitting
  split = csv[col].str.split("/", expand=True)
  # adding
  csv[col] = pd.to_numeric(split[0])/pd.to_numeric(split[1])
  return

In [49]:
csv = data
split(csv,"PAT")
split(csv,"FG")
split(csv,"0-19")
split(csv,"20-29")
split(csv,"30-39")
split(csv,"40-49")
split(csv,"50+")

data.head()

,Player,Team,Gms,PAT,FG,0-19,20-29,30-39,40-49,50+,...,0-19_success,0-19_total,20-29_success,20-29_total,30-39_success,30-39_total,40-49_success,40-49_total,50+_success,50+_total
0,Ka'imi FairbairnK. Fairbairn,HOU,16,0.951220,0.880952,1.0,1.0,1.0,0.800000,0.666667,...,1,1,9,9,11,11,12,15,4,6
1,Justin TuckerJ. Tucker,BAL,16,0.972973,0.897436,NaN,1.0,1.0,0.818182,0.714286,...,0,0,12,12,9,9,9,11,5,7
2,Harrison ButkerH. Butker,KC,16,0.942029,0.888889,NaN,1.0,1.0,0.857143,0.500000,...,0,0,9,9,7,7,6,7,2,4
3,Wil LutzW. Lutz,NO,16,0.981132,0.933333,1.0,1.0,1.0,0.916667,0.666667,...,1,1,5,5,9,9,11,12,2,3
4,Stephen GostkowskiS. Gostkowski,NE,16,0.980000,0.843750,NaN,1.0,1.0,0.666667,0.400000,...,0,0,11,11,10,10,4,6,2,5
